In [3]:
%matplotlib inline

In [4]:
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

In [5]:
from scipy import stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_context("poster")

In [31]:
# A whole world of potential analysis...
kobe = pd.read_csv('data/data.csv')
kobe['known_data'] = kobe['shot_made_flag'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [32]:
print len(kobe)

30697


In [ ]:
kobe.head()

In [33]:
kobe_coltypes = pd.DataFrame(kobe.dtypes.reset_index())
kobe_coltypes.columns = ['col','type']
kobe_coltypes

,col,type
0,action_type,object
1,combined_shot_type,object
2,game_event_id,int64
3,game_id,int64
4,lat,float64
5,loc_x,int64
6,loc_y,int64
7,lon,float64
8,minutes_remaining,int64
9,period,int64


Attempt 1 - Lets just take the average for each known shot up to that shot
--

In [40]:
kobe['rolling_perc'] = pd.rolling_mean(kobe['shot_made_flag'],window=len(kobe['shot_made_flag']),min_periods=1).fillna(0.5)

/Users/apatterson/Envs/kaggles/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=1,window=30697,center=False).mean()
  if __name__ == '__main__':


In [58]:
kobe_output = pd.DataFrame(kobe['rolling_perc'][kobe['known_data']==0])
kobe_output['id'] = kobe_output.index
kobe_output['id'] = kobe_output['id']+1
kobe_output.columns = ['shot_made_flag','shot_id']

In [59]:
kobe_output[['shot_id','shot_made_flag']].to_csv('data/first_attempt.csv',index=False)

Submission logloss: 0.68855
    
Better than 50% mark but not by much!

Attempt 2 - Lets take previous attempt for small sample (Anything with less than 200) and then an incremental naive bayes for the rest
----

In [4]:
from sklearn.naive_bayes import BernoulliNB

In [6]:
#kobe_id = pd.read_csv('data/kobe_x_id.csv')
kobe_x = pd.read_csv('data/kobe_x.csv')
kobe_y = pd.read_csv('data/kobe_y.csv')

In [14]:
from sklearn.decomposition import PCA

comps = PCA(n_components=20)
comps.fit(kobe_x)
print [round(x,2) for x in comps.explained_variance_ratio_]


comps = PCA(n_components=5)
comps.fit(kobe_x)
kobe_x_transformed = comps.transform(kobe_x)

kobe_x_transformed[0:5]

kobe_transformed_df = pd.DataFrame(kobe_x_transformed,columns=['pca1','pca2','pca3','pca4','pca5'])

kobe_transformed_df.to_

[0.44, 0.39, 0.11, 0.03, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


,pca1,pca2,pca3,pca4,pca5
0,-21.713222,2.943203,-4.013827,6.522043,5.059306
1,-21.411333,7.813444,-0.830309,6.254586,5.008957
2,-22.887300,-15.065320,-2.814929,6.372426,1.932314
3,-23.295154,-21.809740,-9.052394,6.855058,1.033232
4,-21.317453,10.247093,14.230525,4.922732,0.623437


In [ ]:
kobe_x.values.tolist()

In [ ]:
from sklearn.naive_bayes import BernoulliNB

i=0
score = []
bnb = BernoulliNB()
new_scores = []
for x, y in zip(kobe_x.iterrows(),kobe_y.iterrows()):
    if i==0:
        #tx = x
        #ty = y
        #print x[1]
        #print y[1]['shot_made_flag']
        #print pd.notnull(y[1]['shot_made_flag'])
        #break
        score.append([i,0.5])
        i+=1
    elif pd.notnull(y[1]['shot_made_flag']):
        bnb.partial_fit([x[1].tolist()],[y[1]['shot_made_flag']],classes=[0,1])
        i+=1
    else:
        score.append([i,bnb.predict_proba(x[1])[0][1]])
        i+=1

/Users/apatterson/Envs/kaggles/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/apatterson/Envs/kaggles/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/apatterson/Envs/kaggles/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if i

In [ ]:
bnb = BernoulliNB()

In [28]:
print tx[1].tolist()

[33.972299999999997, -118.1028, 10.0, 1.0, 0.0, 27.0, 18.0, 2000.0, 10.0, 44.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [29]:
bnb = BernoulliNB()
bnb.partial_fit([tx[1].tolist()],[0],classes=[0,1])

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [ ]:
score = []

for i in test_x.index:
    #print i, train_y[i]
    if i == 0:
        sc = 0.5
        score.append([i,0.5])
    elif i > 0 and i < 200:
        temp_x = train_x[train_x.index< i]
        temp_y = train_y['shot_made_flag'][0:len(temp_x)]
        sc = sum(temp_y)/len(temp_y)
        score.append([i,sum(temp_y)/len(temp_y)])
    else:
        temp_x = train_x[train_x.index< i]
        temp_y = train_y['shot_made_flag'][0:len(temp_x)]
        lrm = LogisticRegression()
        lrm.fit(temp_x,temp_y)
        sc = lrm.predict_proba(test_x[test_x.index==i])[0][1]
    print [i,sc]
    score.append([i,sc])

print score

[0, 0.5]
[7, 0.5]
[16, 0.42857142857142855]
[19, 0.4375]
[32, 0.39285714285714285]
[33, 0.39285714285714285]
[34, 0.39285714285714285]
[35, 0.39285714285714285]
[36, 0.39285714285714285]
[37, 0.39285714285714285]
[44, 0.44117647058823528]
[49, 0.47368421052631576]
[54, 0.42857142857142855]
[59, 0.45652173913043476]
[65, 0.45098039215686275]
[66, 0.45098039215686275]
[70, 0.46296296296296297]
[79, 0.45161290322580644]
[84, 0.43939393939393939]
[85, 0.43939393939393939]
[94, 0.47297297297297297]
[103, 0.48780487804878048]
[112, 0.45555555555555555]
[122, 0.45454545454545453]
[125, 0.46534653465346537]
[132, 0.46728971962616822]
[140, 0.45614035087719296]
[143, 0.44827586206896552]
[149, 0.45454545454545453]
[151, 0.45901639344262296]
[152, 0.45901639344262296]
[155, 0.45161290322580644]
[158, 0.44444444444444442]
[164, 0.4580152671755725]
[171, 0.45255474452554745]
[181, 0.43835616438356162]
[191, 0.43870967741935485]
[192, 0.43870967741935485]
[195, 0.43949044585987262]
[201, 0.17272435

In [ ]:
with open('data/attempt_2_submission.csv','w') as f:
    f.write('shot_id,shot_made_flag'+'\n')
    for line in score:
        f.write(','.join([str(line[0]+1),str(line[1])])+'\n')